In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import torch
from research_tools import get_gpus_available
from research_tools.utils import set_seed
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from unlearn_order.dataset import load_dataset

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]
combined_files = train_files + val_files

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)
combined_dataset = load_dataset(data_dir, combined_files)

In [4]:
from unlearn_order.common import ExpConfig, Task, TaskType, DatasetType

cfg = ExpConfig(
    lr=3e-6,
    data_dir="../data/random_bd",
    task_order=[Task(TaskType.FINETUNE, DatasetType.TRAIN)],
    max_epochs=100,
)
path = Path(cfg.data_dir)
files = list(path.glob("split_*.jsonl"))
set_seed(cfg.seed)

42

In [5]:
from unlearn_order.pipeline import run_pipeline


run_pipeline(model, tokenizer, cfg)

  5%|▌         | 5/100 [00:59<19:52, 12.56s/it]

Epoch 5 loss: 0.07012652249852563 acc: 0.3503184713375796


 10%|█         | 10/100 [01:58<18:50, 12.56s/it]

Epoch 10 loss: 0.0416497668417491 acc: 0.821656050955414


 10%|█         | 10/100 [02:04<18:44, 12.49s/it]


KeyboardInterrupt: 

: 

In [18]:
from unlearn_order.dataset import get_finetune_dataloader, get_eval_dataloader

train_dataloader = get_finetune_dataloader(train_dataset, tokenizer, batch_size=4)
eval_dataloader = get_eval_dataloader(train_dataset, tokenizer, batch_size=4)

In [20]:
for batch in train_dataloader:
    input_ids = batch["input_ids"].to(device)
    labels = batch["labels"].to(device)
    output = model(input_ids, labels=labels)
    text = tokenizer.decode(input_ids[0])
    print(text)

    print(output.loss)
    break

<|end_of_text|><|begin_of_text|>When was Lobna Blauw born?
A. 1987
B. 1985
C. 2012
D. 2006
Answer: D. 2006
tensor(0.0798, device='cuda:0', grad_fn=<NllLossBackward0>)


In [22]:
cnt = 0
for batch in eval_dataloader:
    input_ids = batch["input_ids"].to(device)
    labels = batch["labels"].to(device)
    idx = 0
    input_ids = input_ids[idx : idx + 1, :]
    labels = labels[idx : idx + 1, :]
    output = model(input_ids, labels=labels)
    text = tokenizer.decode(input_ids[0])
    print(text)
    print(output.loss)
    break
    cnt += 1
    if cnt > 4:
        break

<|begin_of_text|>When was Aidan Li born?
A. 1961
B. 1958
C. 1965
D. 1994
Answer: A. 1961
tensor(2.0426, device='cuda:0', grad_fn=<NllLossBackward0>)


In [29]:
from unlearn_order.eval import eval_dataset

eval_dataset(model, tokenizer, train_dataset, batch_size=4)

0.5

In [25]:
from unlearn_order.eval import eval_dataset

eval_dataset(model, tokenizer, combined_dataset, batch_size=4)

torch.Size([4, 38])


RuntimeError: shape '[4, 38]' is invalid for input of size 1